In [1]:
import requests
import pandas as pd

import tqdm

import random
import time

from bs4 import BeautifulSoup

from os import path

In [2]:
tipo_imovel = "apartamentos"

In [3]:
if path.exists(f"data/{tipo_imovel}_completo_intermediario.csv"):
    df = pd.read_csv(f"data/{tipo_imovel}_completo_intermediario.csv")
else:
    df = pd.read_csv(f"data/{tipo_imovel}_incompleto.csv")
    df['condominio'] = ''
    df['IPTU'] = ''
    df['lat'] = ''
    df['long'] = ''
    df['area_total'] = ''
    df['area_util'] = ''
    df['banheiros'] = ''
    df['vagas'] = ''
    df['quartos'] = ''
    df['suites'] = ''
    df['idade'] = ''
    df['endereco'] = ''
    df['bairro'] = ''
    df['idade_anuncio'] = ''
    df['downloaded'] = False

print(df.shape)
df.head()

(23521, 18)


,nome,valor_venda,url,condominio,IPTU,lat,long,area_total,area_util,banheiros,vagas,quartos,suites,idade,endereco,bairro,idade_anuncio,downloaded
0,4 Quartos 180 m² - Península - Reformado - Nas...,1150000,https://www.wimoveis.com.br/propriedades/4-qua...,R$ 1.050,R$ 1.300,-15.833316,-48.045697,205m²,180m²,4.0,3.0,4.0,2.0,3.0,"Av das araucarias, Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",119,True
1,Vendo Lido Apartamento De Um Quarto Andar Alto!,230000,https://www.wimoveis.com.br/propriedades/vendo...,R$ 320,NaN,-15.842480,-48.030153,42m²,42m²,1.0,1.0,1.0,0.0,2.0,"R. 25 Sul, Lote 17 - Aguas Claras, Aguas Sul...",", Aguas Sul, Águas Claras",59,True
2,"Águas Claras, Ed. Conquest, Apartamento, Garag...",198000,https://www.wimoveis.com.br/propriedades/aguas...,R$ 234,NaN,-15.829953,-48.033735,NaN,36m²,1.0,1.0,1.0,NaN,11.0,"Quadra Quadra 107 0, Aguas Norte, Águas Claras",", Aguas Norte, Águas Claras",mais de 1 ano\n,True
3,"Apartamentos 4 Quartos (Suítes), Noroeste - Pa...",1940183,https://www.wimoveis.com.br/propriedades/apart...,NaN,NaN,-15.748282,-47.915743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,"Qmsw 05, Ed. Mont Blanc. Kit Nascente Com Gara...",220000,https://www.wimoveis.com.br/propriedades/qmsw-...,R$ 260,NaN,-15.795590,-47.929983,NaN,25m²,1.0,1.0,1.0,0.0,10.0,"QMSW 05 Lt 02 Blocos A B, Sudoeste, Brasília",", Sudoeste, Brasília",90,True


In [4]:
SAVE_EVERY_N = 100

In [5]:
for i, row in tqdm.tqdm_notebook(df.iterrows(), total=df.shape[0]):
    
    if(row.downloaded == True):
        print(f"item {row.url} ja baixado, passando")
        continue
    
    try:
        
        tqdm.tqdm.write(str(i + 1) +" Buscando item " + row.url)
        response = requests.get(row.url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Condominio e IPTU
        valores = soup.findAll("div",{"class":"block-expensas block-row"})
        if(len(valores)>0):
            for item in valores:
                #tqdm.tqdm.write(str(item))
                if("Condomínio" in str(item) ):
                    #print(item.find("span").text)
                    df.at[i,'condominio'] = item.find("span").text
                else:
                    df.at[i,'IPTU'] = item.find("span").text
                    
        # latlong
        google_maps_url = soup.find("img", {"class":"static-map"})['src']
        
        lat_long = google_maps_url.split("?center=")[1].split("&zoom=")[0]
        
        lat = lat_long.split(",")[0]
        df.at[i,'lat'] = lat
        
        long = lat_long.split(",")[1]
        df.at[i,'long'] = long
        
        # icones
        icon_total = soup.find("i",{"class":"icon-f icon-f-stotal"})
        if icon_total is not None:
            area_total = icon_total.findNext('b').text
            df.at[i,'area_total'] = area_total
        
        icon_util = soup.find("i",{"class":"icon-f icon-f-scubierta"})
        if icon_util is not None:
            area_util = icon_util.findNext('b').text
            df.at[i,'area_util'] = area_util
            
        icon_banheiro = soup.find("i",{"class":"icon-f icon-f-bano"})
        if icon_banheiro is not None:
            qtd_banheiros = icon_banheiro.findNext('b').text
            df.at[i,'banheiros'] = qtd_banheiros
            
        icon_vagas = soup.find("i",{"class":"icon-f icon-f-cochera"})
        if icon_vagas is not None:
            qtd_vagas = icon_vagas.findNext('b').text
            df.at[i,'vagas'] = qtd_vagas
        
        icon_quartos = soup.find("i",{"class":"icon-f icon-f-dormitorio"})
        if icon_quartos is not None:
            qtd_quartos = icon_quartos.findNext('b').text
            df.at[i,'quartos'] = qtd_quartos
        
        icon_suites = soup.find("i",{"class":"icon-f icon-f-toilete"})
        if icon_suites is not None:
            qtd_suites = icon_suites.findNext('b').text
            df.at[i,'suites'] = qtd_suites
        
        icon_idade = soup.find("i",{"class":"icon-f icon-f-antiguedad"})
        if icon_idade is not None:
            idade = icon_idade.findNext('b').text
            df.at[i,'idade'] = idade
        
        h2_endereco = soup.find("h2",{"class":"title-location"})
        if h2_endereco is not None:
            df.at[i,'endereco'] = h2_endereco.text
            df.at[i,'bairro'] = h2_endereco.findNext("span").text
        
        h5_idade = soup.find("h5",{"class":"section-date css-float-r"})
        if h5_idade is not None:
            df.at[i,'idade_anuncio'] = h5_idade.text.replace("\n Publicado há ","").replace(" dias\n","")
        
    except Exception as e:
        print("exceção ao buscar dados. Mensagem:", str(e))
        pass
    
    df.at[i,'downloaded'] = True
    
    if (i % SAVE_EVERY_N) == 0:
        print(f"Salvando arquivo intermediario em data/{tipo_imovel}_completo_intermediario.csv")
        df.to_csv(f"data/{tipo_imovel}_completo_intermediario.csv", index=False)
    
    #dorme
    aleatorio = random.randint(1, 4)
    tqdm.tqdm.write("agora vou aguardar: " + str(aleatorio))
    time.sleep(aleatorio)

print(df.shape)
df.head(10)

item https://www.wimoveis.com.br/propriedades/4-quartos-180-m-sup2--peninsula-reformado-2945740538.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/vendo-lido-apartamento-de-um-quarto-andar-alto!-2947054048.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/aguas-claras-ed.-conquest-apartamento-garagem-2939464716.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-4-quartos-suites-noroeste-parque-das-2936585040.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/qmsw-05-ed.-mont-blanc.-kit-nascente-com-garagem-2946358648.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/arlindo-vende-apto-02-quartos-suite-armarios-2946255247.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/real-paris-com-03-quartos-e-02-vagas-de-garagem-a-2947679427.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-4-quartos-suites-noroeste-2944

item https://www.wimoveis.com.br/propriedades/qd-101-lt-8-2938480996.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/altos-taguatinga-2-quartos-pronto-morar-mcmv-2930896037.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/cobertura-3-quartos-park-sul-prime-residence-vista-2945518590.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-com-3-suites-plenas-churrasq-a-carvao-2941839531.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/asa-norte-3-quartos-3-andar-vista-livre-area-2941451199.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqsw-301-sonata-canto-nascente-vista-libre-2945230083.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/gavino-99141-0147-aceita-troca-por-apt-de-menor-2947637053.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-no-brisas-do-lago-locado-por-515-mil.-2945799048.html ja baixado, p

item https://www.wimoveis.com.br/propriedades/apartamento-3-quartos-noroeste-pronto-pra-morar-2947424717.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/116-innovatore-avalia-permuta-asa-sul-2943085202.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqn-404-3-quartos-reformado-localizacao-2946508198.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/nascente-na-sqs-402-bloco-n-com-03-quartos-a-venda-2942609793.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/aguas-claras-residencial-castanheiras-3-quartos-1-2945793586.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/quadra-210-apartamento-3-quartos-reformado-2946815902.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/bahamas-av-jequitiba-kitnet-com-vaga-de-garagem-2947741527.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/o4-quartos-com-127-m-sup2--02-vagas-predio-novo!-294726

agora vou aguardar: 3
2192 Buscando item https://www.wimoveis.com.br/propriedades/sqnw-307-cobertura-duplex-312-m-2947521481.html
agora vou aguardar: 1
2193 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-02-quartos-c-suite-01-vaga-apenas-10-2944738360.html
agora vou aguardar: 2
2194 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-samambaia-norte-2943065692.html
agora vou aguardar: 3
2195 Buscando item https://www.wimoveis.com.br/propriedades/2-quartos-desocupado-cnb-4-2946122525.html
agora vou aguardar: 3
2196 Buscando item https://www.wimoveis.com.br/propriedades/costa-azul-02-quartos-01-suite-por-apenas-r$260.000-2946564379.html
agora vou aguardar: 4
2197 Buscando item https://www.wimoveis.com.br/propriedades/nascente-andar-alto-e-vista-livre-2946754441.html
agora vou aguardar: 2
2198 Buscando item https://www.wimoveis.com.br/propriedades/sqs-307-04-quartos-02-vagas-andar-alto-vazado-2942990735.html
agora vou aguardar: 1
2199 Buscando item https

item https://www.wimoveis.com.br/propriedades/excelente-vazado-com-03-quartos-na-sqs-410-bloco-e-a-2943768387.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-03-quartos-c-01-suite-01-vaga-2944702955.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/quadra-102-andar-alto!-vista-livre!-nascente!-98311-2946287372.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/all-reformado-98243-8383-2947568933.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-2-quartos-samambaia-norte-2934435135.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqs-104-amplitude-maravilhoso-apartamento-2942458841.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-3-suites-3-quartos-setor-alto-da-gloria-2946961628.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/residencial-ape-exclusivo-2-quartos-2-vagas-62-2945685008.htm

...

item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-no-top-life-long-beach-em-2945413315.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/vive-la-vie-nascente-reformado-02-vagas-soltas-2947521911.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-1-e-2-quartos-lago-norte-liberty-lago-2934273317.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/quadra-14-ed.-cidade-serrana-apartamento-com-2-2945213639.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqnw-311-lancamento-noroeste-2947459765.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqnw-106-vazado-nascente-164-11-m-sup2--apenas-48-2947521675.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-4-quartos-suites-noroeste-311-by-2947427315.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/quadra-301-residencial-beatriz-apartamento-de-2-2947161931

item https://www.wimoveis.com.br/propriedades/apartamentos-4-quartos-suites-noroeste-parque-das-2936585040.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/life-resort-2-suites.-vazado.-andar-alto.-2941263714.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/reformado-nascente-vista-livre-cobertura-2947080344.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/shcgn-716-lindo-2-quartos-reformado-rico-em-armarios-e-2947742508.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-2-quartos-noroeste-jardins-das-2943013987.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/linda-cobertura-de-3-quartos-200-m-sup2--nascente-3-2947758196.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-no-top-life-long-beach-em-2945413315.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/vive-la-vie-nascente-reformado-02-vagas-sol

item https://www.wimoveis.com.br/propriedades/rua-24-norte-loft-andar-alto-nascente-e-2-vagas-2946309846.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqsw-105-andar-alto-reformado!-2945141013.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-2-quartos-noroeste-jardins-dos-lirios-2936130912.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-duplex-2-qts-110-m-sup2--decorado-2947166047.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqs-310-5-quartos-reformado-mobiliado-2946257818.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/villa-borgio-belissimo-apartamento-4-quartos-em-aguas-2947742589.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-2-quartos-noroeste-jardins-das-2943013987.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqs-105-andar-alto-nascente-vista-livre-vazado-2947355795.htm

item https://www.wimoveis.com.br/propriedades/apartamento-cobertura-asa-norte-sqn-311-alto-2947524973.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-4-quartos-suites-noroeste-2944959225.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/03-quartos-102-60-m-sup2--vista-para-piscina-2938362230.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-66-m-sup2--2-quartos-1-suite-e-1-vaga-de-2943701039.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/-61-9.99810736-entrada-de-r$37.500-direto-com-a-2945459841.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamentos-no-noroeste-311-by-lotus-2947544640.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/peninsula-lindo-reformado-03-vagas-soltas-2943989892.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-2-quartos-a-venda-2947749831.html ja baixado, passa

,nome,valor_venda,url,condominio,IPTU,lat,long,area_total,area_util,banheiros,vagas,quartos,suites,idade,endereco,bairro,idade_anuncio,downloaded
0,4 Quartos 180 m² - Península - Reformado - Nas...,1150000,https://www.wimoveis.com.br/propriedades/4-qua...,R$ 1.050,R$ 1.300,-15.833316,-48.045697,205m²,180m²,4.0,3.0,4.0,2.0,3.0,"Av das araucarias, Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",119,True
1,Vendo Lido Apartamento De Um Quarto Andar Alto!,230000,https://www.wimoveis.com.br/propriedades/vendo...,R$ 320,NaN,-15.842480,-48.030153,42m²,42m²,1.0,1.0,1.0,0.0,2.0,"R. 25 Sul, Lote 17 - Aguas Claras, Aguas Sul...",", Aguas Sul, Águas Claras",59,True
2,"Águas Claras, Ed. Conquest, Apartamento, Garag...",198000,https://www.wimoveis.com.br/propriedades/aguas...,R$ 234,NaN,-15.829953,-48.033735,NaN,36m²,1.0,1.0,1.0,NaN,11.0,"Quadra Quadra 107 0, Aguas Norte, Águas Claras",", Aguas Norte, Águas Claras",mais de 1 ano\n,True
3,"Apartamentos 4 Quartos (Suítes), Noroeste - Pa...",1940183,https://www.wimoveis.com.br/propriedades/apart...,NaN,NaN,-15.748282,-47.915743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,"Qmsw 05, Ed. Mont Blanc. Kit Nascente Com Gara...",220000,https://www.wimoveis.com.br/propriedades/qmsw-...,R$ 260,NaN,-15.795590,-47.929983,NaN,25m²,1.0,1.0,1.0,0.0,10.0,"QMSW 05 Lt 02 Blocos A B, Sudoeste, Brasília",", Sudoeste, Brasília",90,True
5,Arlindo Vende Apto 02 Quartos - Suíte - Armári...,485000,https://www.wimoveis.com.br/propriedades/arlin...,NaN,R$ 737,-15.862571,-48.028088,NaN,87m²,2.0,1.0,2.0,1.0,5.0,"Rua Babaçu 03, Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",95,True
6,Real Paris Com 03 Quartos E 02 Vagas De Garage...,525000,https://www.wimoveis.com.br/propriedades/real-...,R$ 632,R$ 925,-15.836555,-48.010368,140m²,79m²,2.0,2.0,3.0,1.0,10.0,"Rua 3 4, Aguas Norte, Águas Claras",", Aguas Norte, Águas Claras",14,True
7,"Apartamentos 4 Quartos (Suítes), Noroeste - Re...",3181500,https://www.wimoveis.com.br/propriedades/apart...,NaN,NaN,-15.742127,-47.910671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
8,Centrale De Canto 64 m² Vendo Ou Troco Por Apt...,350000,https://www.wimoveis.com.br/propriedades/centr...,NaN,NaN,-15.838311,-48.023134,62m²,62m²,2.0,0.0,2.0,1.0,6.0,"Rua 18 , Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",104,True
9,Sqn 415 / Exclusivo / 3 Qts Suíte E Closet / A...,735000,https://www.wimoveis.com.br/propriedades/sqn-4...,NaN,NaN,-15.739217,-47.886586,NaN,78m²,0.0,0.0,3.0,3.0,5.0,"SQN 415 , Asa Norte, Brasília",", Asa Norte, Brasília",63,True


In [6]:
df.to_csv(f"data/{tipo_imovel}_completo.csv", index=False)

In [8]:
df.isna().sum()

nome                 0
valor_venda          0
url                  0
condominio       10319
IPTU             18020
lat               2167
long              2167
area_total       11497
area_util         3781
banheiros         4168
vagas             4752
quartos           3467
suites            5703
idade            10588
endereco          4223
bairro            4223
idade_anuncio     4225
downloaded           0
dtype: int64